## Initialisation

In [ ]:
# Check for running GPU
!nvidia-smi

In [ ]:
# Run once for initial installs
#! pip install farm-haystack
! pip install git+https://github.com/deepset-ai/haystack.git
! pip install urllib3==1.25.4
#! pip install torch==1.6.0+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers[torch]
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html

In [ ]:
# Run this if want to see graphic representation of pipeline 
! apt install libgraphviz-dev graphviz
! pip install pygraphviz

In [29]:
# Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2
! sleep 30

In [1]:
# Connect to Elasticsearch - Used for Elastic Retriever 
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)
                  )
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store_ES = ElasticsearchDocumentStore(similarity="dot_product", host="localhost", username="", password="")

02/26/2021 21:43:56 - INFO - faiss.loader -   Loading faiss with AVX2 support.
02/26/2021 21:43:56 - INFO - faiss.loader -   Loading faiss.
02/26/2021 21:43:58 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
02/26/2021 21:43:58 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.006s]
02/26/2021 21:43:58 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.007s]
02/26/2021 21:43:58 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.004s]
02/26/2021 21:43:58 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.014s]
02/26/2021 21:43:58 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.004s]


In [2]:
from typing import List
import requests
import pandas as pd
from haystack import Document
from haystack.document_store.faiss import FAISSDocumentStore

# FAISS DocumentStore used for DPR and embedding retriever
document_store_FAISS = FAISSDocumentStore(
    faiss_index_factory_str="Flat",
    return_embedding=True
)

In [3]:
import pprint
pp = pprint.PrettyPrinter(indent=2)

## Document Preprocessing

In [4]:
from haystack.reader.farm import FARMReader
import haystack

converter = haystack.file_converter.txt.TextConverter(
                    remove_numeric_tables=False,
                    valid_languages = ["en"])

as4 = converter.convert(file_path="/content/as4-winterBarley.txt")


processor = haystack.preprocessor.preprocessor.PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="passage",
    split_length=1,
    split_respect_sentence_boundary=False,
    split_overlap=0
)

as4Docs = processor.process(as4)

for i in range(len(as4Docs)):
    as4Docs[i]["meta"]["table"] = False

document_store_ES.delete_all_documents()
document_store_ES.write_documents(as4Docs)
document_store_FAISS.delete_all_documents()
document_store_FAISS.write_documents(as4Docs)

#backagain = document_store.get_all_documents();

for i in range(0,len(as4Docs)):
    print(str(i) + ":", end = " ")
    print(as4Docs[i])

02/26/2021 21:44:05 - INFO - elasticsearch -   POST http://localhost:9200/document/_delete_by_query [status:200 request:0.049s]
02/26/2021 21:44:07 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.873s]


0: {'text': 'Three-spray programmes are recommended for winter barley due to its response to T3 fungicides and the rise in severity of late season ramularia. As such T1 and T2 can be applied slightly earlier if required to better match up with PGR timings.', 'meta': {'_split_id': 0, 'table': False}}
1: {'text': 'The main fungicide used for winter barley disease control is Prothioconazole; however growers should look to protect this active by including alternative modes of action.', 'meta': {'_split_id': 1, 'table': False}}
2: {'text': 'For winter barley, timing for T1 (GS30-31) is important due to more GAI on lower than on upper leaves. The slightly earlier timing will allow better pairing with an early growth regulator and reduce the need for a T0 treatment.', 'meta': {'_split_id': 2, 'table': False}}
3: {'text': 'For winter barley, T1 (GS30-31) application options are:\nSiltra Xpro 0.4 -0.6 l/ha\nCebara 1.0 -1.5 l/ha + Proline 0.25 l/ha,\nComet 0.4 l/ha + Proline 0.3-0.4 l/ha\nFandan

In [5]:
# Update table content and table description - csv for table content, txt for table description
# The files are under the /tables directory, the name does not matter, as long as the csv and the txt match each other.
# Need to rerun this section when new table is uploaded.

import csv
import os

data = []
docs= []

for file in [file for file in os.listdir(os.getcwd()+"/tables") if ".csv" in file]:
  with open(os.getcwd()+"/tables/"+file, mode='r') as infile:
    reader = csv.reader(infile)
    new_dict = {row[0]:row[1:] for row in reader}
    data.append(new_dict)
  infile.close()
  with open(os.getcwd()+"/tables/"+file[:-4]+".txt", mode='r') as infile:
    docs.append(infile.read())
  infile.close()

with open('table_text.txt', 'w') as outfile:
    for item in docs:
        outfile.write("%s\n\n" % item)
outfile.close()

# Construct FAISS DocumentStore for table content

tables = converter.convert(file_path="/content/table_text.txt")

tableDocs = processor.process(tables)
for i in range(len(tableDocs)):
  tableDocs[i]["meta"]["index"] = i
  tableDocs[i]["meta"]["table"] = True

document_store_ES.write_documents(tableDocs)
document_store_FAISS.write_documents(tableDocs)

02/26/2021 21:44:10 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.525s]


In [6]:
# NLP based keyword extractor
# New category upload is not handled here
from KeyInfoExtractor import KeywordExtractor
key_word_extractor = KeywordExtractor()
key_word_extractor.read_key_info_file("categories.csv")
key_word_extractor.check_current_categories(True)
questions = {}
for ki in key_word_extractor.key_infos:
  questions[ki.name] = ki.question
pp.pprint(questions)

Currently held Categories: 
*  Crop:  ['wheat', 'oats', 'rape', 'peas', 'spring', 'beans', 'barley', 'linseed', 'winter', 'oilseed']  ||  ['', 'spring oilseed rape', 'winter oats', 'winter wheat', 'spring oats', 'winter oilseed rape', 'spring beans', 'spring linseed', 'winter linseed', 'spring wheat', 'spring barley', 'spring peas', 'winter barley', 'winter beans']
*  Disease:  ['mildew', 'rhynchosporium', 'brown', 'ear', 'ramularia', 'light', 'botrytis', 'fusarium', 'blight', 'phoma', 'blotch', 'crown', 'eyespot', 'rust', 'canker', 'alternaria', 'chocolate', 'stem', 'leaf', 'downy', 'mycosphaerella', 'septoria', 'verticillium', 'spot', 'sclerotinia', 'net', 'drechslera', 'aschochyta', 'microdochium', 'tan', 'yellow']  ||  ['mildew', 'rhynchosporium', '', 'ramularia', 'botrytis', 'brown rust', 'fusarium', 'light leaf spot', 'yellow rust', 'eyespot', 'drechslera leaf spot', 'phoma leaf spot', 'downy mildew', 'crown rust', 'rust', 'ear blight', 'alternaria', 'mycosphaerella', 'tan spot',

In [7]:
# Preliminary parser for questions - specially tackles the variety of the crops
# New json upload is not handled here
from parsing import Parser

parser = Parser()

# To parse question into general case: e.g. what to apply to Dunston -> what to apply to winter wheat
def question_parsing(text):
  parsed = parser.parse(text)
  parsed_string = ""
  for string in parsed["simple"]:
    parsed_string = parsed_string + string
  parsed_string = re.sub("[\(\[].*?[\)\]]", "", parsed_string)
  parsed_string = re.sub("  ", " ", parsed_string)
  return parsed_string

# To extract the keywords that the extractor cannot handle - e.g. variety
def generate_keywords(text):
  parsed = parser.parse(text)
  parsed_string = ""
  for string in parsed["simple"]:
    parsed_string = parsed_string + string
  keywords = {}
  while (parsed_string.find("(") != -1):
    start = parsed_string.find("(")
    end = parsed_string.find(")")
    bracket = parsed_string[start+1:end]
    if bracket.find(":") != -1:
      colon = bracket.find(":")
      keywords[bracket[1:colon].replace(" ", "")] = bracket[colon+1:].replace(" ", "")
    parsed_string = parsed_string[end+1:] 
  return keywords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# Pressure score system when crops with different pressure score need to be treated differently.
# Need to rerun update pressure table if new pressure table is uploaded (need to include the origianl csv content)

import csv
import re

class PressureScoreGenerator:
  def __init__(self):
    self.pressure_table = {}

  def update_pressure_table(self,filename='pressure_score.csv'):
    with open('pressure_score.csv', mode='r') as infile:
        reader = csv.reader(infile)
        for row in reader:
          if row[0] in self.pressure_table:
            if row[1] in self.pressure_table[row[0]]:
              self.pressure_table[row[0]][row[1]][len(self.pressure_table[row[0]][row[1]])+1] = [row[2],int(row[3])]
            else:
              self.pressure_table[row[0]][row[1]] = {1:[row[2],int(row[3])]}
          else:
            self.pressure_table[row[0]]={row[1]:{1:[row[2],int(row[3])]}}
    infile.close()

  def calculate_pressure_score(self,crop):
    if crop in self.pressure_table:
      print("To give a better suggestion. We need to calculate the disease pressure of your current case. Please specify your choice by replying with the number accordingly.")
      score = 0
      # Would it be possible to change this into options by click?
      for key in self.pressure_table[crop].keys():
        print("Please choose the " + key + " which best describes your current case: ")
        for option in self.pressure_table[crop][key]:
          print(option, ": ",self.pressure_table[crop][key][option][0])
        choice = input()
        while not choice.isnumeric() or not (int(choice) >= 1 and int(choice) <= len(self.pressure_table[crop][key])):
          choice = input("Your input is not Valid. Please check and try again.")
        score = score + self.pressure_table[crop][key][int(choice)][1]
      if score <= 12:
        pressure_level = "low"
      elif score <= 15:
        pressure_level = "moderate"
      elif score <= 18:
        pressure_level = "high"
      else:
        pressure_level = "very high"
      print("Your current disease pressure score is " + str(score) + ".\nThis means that the disease pressure is " + pressure_level + ".")
      return pressure_level
    else:
      return -1

pressure_score_generator = PressureScoreGenerator()
pressure_score_generator.update_pressure_table()
#pp.pprint(pressure_score_generator.pressure_table)

## Building Individual Components

In [11]:
# Naive retriver based on tf * idf - Default BM25, can be cunstomised
from haystack.retriever.sparse import ElasticsearchRetriever
es_retriever = ElasticsearchRetriever(document_store=document_store_ES)

In [12]:
# Alternative retriever - double BERT neural networks for question and doc embedding
from haystack.retriever.dense import DensePassageRetriever
dpr_retriever = DensePassageRetriever(
    document_store=document_store_FAISS,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True,
    embed_title=True,
)

In [13]:
document_store_FAISS.update_embeddings(retriever=dpr_retriever) #possible training of dpr model

02/26/2021 21:44:57 - INFO - haystack.document_store.faiss -   Updating embeddings for 31 docs...
Creating Embeddings: 100%|██████████| 2/2 [00:00<00:00,  3.94 Batches/s]
10000it [00:00, 18439.36it/s]


In [14]:
# Alternative retriever - single BERT to embed both question and doc, may be better for similar documents (our case)
from haystack.retriever.dense import EmbeddingRetriever
embedding_retriever = EmbeddingRetriever(document_store=document_store_FAISS,
                               embedding_model="deepset/sentence_bert")

02/26/2021 21:44:57 - INFO - haystack.retriever.dense -   Init retriever using embeddings of model deepset/sentence_bert
02/26/2021 21:44:57 - INFO - farm.utils -   Using device: CUDA 
02/26/2021 21:44:57 - INFO - farm.utils -   Number of GPUs: 1
02/26/2021 21:44:57 - INFO - farm.utils -   Distributed Training: False
02/26/2021 21:44:57 - INFO - farm.utils -   Automatic Mixed Precision: None
02/26/2021 21:45:00 - WARNING - farm.utils -   Failed to log params: Changing param values is not allowed. Param with key='prediction_heads' was already logged with value='TextSimilarityHead' for run ID='a1fe11a62a92431a971f12ad6c5013a9'. Attempted logging new value ''.
02/26/2021 21:45:02 - WARNING - farm.utils -   Failed to log params: Changing param values is not allowed. Param with key='processor' was already logged with value='TextSimilarityProcessor' for run ID='a1fe11a62a92431a971f12ad6c5013a9'. Attempted logging new value 'InferenceProcessor'.
02/26/2021 21:45:02 - WARNING - farm.utils -   

In [15]:
# Reader to further scan with Hugging Face models
# reader = TransformersReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)
# reader = FARMReader(model_name_or_path="deepset/bert-large-uncased-whole-word-masking-squad2", use_gpu=True)

In [16]:
# Decide whether the answer should be retrieved from the tables or the general texts
class QueryClassifier:
  outgoing_edges = 2

  def run(self, **kwargs):
    #print("Running Query Classifier")
    #print(len(kwargs["documents"]))
    #print(kwargs["documents"][0].meta)
    #for doc in kwargs["documents"]:
    #  print(doc.text)
    #print(kwargs["documents"][0].meta["table"] == "0")
    #docs = kwargs["documents"]
    #for doc in docs:
    #  print(doc.text)
    #  print(doc.meta)
    if (kwargs["documents"][0].meta["table"] == "0"):
      #print("It is a general question.")
      return (kwargs, "output_1")
    else:
      #print("It is a table-related question.")
      return (kwargs, "output_2")

query_classifier = QueryClassifier()

In [17]:
# Customised retriever - Process table content using TAPAS
from haystack.retriever.dense import BaseRetriever
from transformers import pipeline, AutoTokenizer, AutoModelForTableQuestionAnswering, TableQuestionAnsweringPipeline

class TableRetriever:
  outgoing_edges = 1

  def __init__(self):
    self.tokenizer = AutoTokenizer.from_pretrained("google/tapas-base-finetuned-wtq")
    self.model = AutoModelForTableQuestionAnswering.from_pretrained("google/tapas-base-finetuned-wtq")
    self.tableQA = TableQuestionAnsweringPipeline(model=self.model,tokenizer=self.tokenizer)

  def run(self,**kwargs):
    #print("Running Table Retriever")
    
    #print("Finished")
    return ({"result":self.tableQA(data[int(kwargs["documents"][0].meta["index"])], kwargs["query"])},"output_1")

table_retriever = TableRetriever()

In [37]:
# Ask further questions if different context is detected.
# In the special case of pressure score, if it can be calculated, a different set of questions will be asked to decide the score.

class FurtherQuestionGenerator:
  outgoing_edges = 1

  def individualFiltersGenerator(self, text):
    text = text.lower()
    keywords = key_word_extractor.get_best_matches(text)
    #print(current_filters)
    #print(key_word_extractor.get_best_matches(text.lower(), give_list=True))
    
    # The extraction of pressure keywords work differently from others
    if text.find("pressure") != -1:
      keywords["pressure"] = []
      if "low" in text[text.find("pressure")-20:text.find("pressure")+30]:
        keywords["pressure"].append("low")
      if "moderate" in text[text.find("pressure")-20:text.find("pressure")+30]:
        keywords["pressure"].append("moderate")
      if "high" in text[text.find("pressure")-20:text.find("pressure")+30]:
        keywords["pressure"].append("high")
      if "very high" in text[text.lower().find("pressure")-20:text.lower().find("pressure")+30] or "extreme" in text[text.find("pressure")-20:text.find("pressure")+30]:
        keywords["pressure"].append("very high")
    
    # To include special keywords such as variety
    parsed_keywords = generate_keywords(text)
    if parsed_keywords != {}:
      for key,value in parsed_keywords.items():
        if key not in keywords:
          keywords[key] = [value]
        else:
          keywords[key].append(value)
    return keywords

  def topDocsFilterGenerator(self, docs):
    return [self.individualFiltersGenerator(doc.text) for doc in docs]

  def filters_difference(self, filters_list, specified = []):
    current_filters = {}
    for filters in filters_list:
      for category, filters in filters.items():
        if ((category not in specified) and (category in current_filters) and (filters != current_filters[category])):
          return category
        elif ((category not in specified) and (category not in current_filters)):
          current_filters[category] = filters
    return None

  def furtherQuestions(self, docs, specified=[], original_filters={}):
    filters_list = self.topDocsFilterGenerator(docs)
    #print(len(docs),len(filters_list))

    # Filter the retrieved docs before asking questions, eliminate docs with different keywords but keep those does not mention the keywords.
    for keyword,new_key in original_filters.items():
      temp_d = [doc for i,doc in enumerate(docs) if not ((keyword in filters_list[i].keys()) and (new_key[0].lower() not in filters_list[i][keyword]))]
      temp_f = [filters for i,filters in enumerate(filters_list) if not ((keyword in filters_list[i].keys()) and (new_key[0].lower() not in filters_list[i][keyword]))]
    docs = temp_d
    filters_list = temp_f
    #print(len(docs),len(filters_list))

    # Rank the docs based on closeness to keywords in question
    match = [0 for doc in docs]
    for keyword,new_key in original_filters.items():
      match = [match[i] + 1 if ((keyword in filters_list[i].keys()) and (new_key[0].lower() in filters_list[i][keyword])) else match[i] for i in range(len(filters_list))]
      match = [match[i] + 1 if ((keyword in filters_list[i].keys()) and (filters_list[i][keyword] == [new_key[0].lower()])) else match[i] for i in range(len(filters_list))]
    
    # Use only questions from the top 3 results. Otherwise too many questions asked.
    docs = [x for _,x in sorted(zip(match,docs), key=lambda pair: pair[0], reverse=True)]
    filters_list = [x for _,x in sorted(zip(match,filters_list), key=lambda pair: pair[0], reverse=True)]
    keyword = self.filters_difference(filters_list[:3],specified)
    contained_pressure = False
    while keyword is not None:
      if keyword != "pressure":
        new_key = input(questions[keyword] + "(If not, please reply No) ")
        if (new_key.lower() != "no"):
          match = [match[i] + 1 if ((keyword in filters_list[i].keys()) and (new_key.lower() in filters_list[i][keyword])) else match[i] for i in range(len(filters_list))]
          match = [match[i] + 1 if ((keyword in filters_list[i].keys()) and (filters_list[i][keyword] == [new_key.lower()])) else match[i] for i in range(len(filters_list))]
        specified.append(keyword)
        original_filters[keyword] = [new_key]
      else:
        contained_pressure = True
        specified.append(keyword)
        original_filters[keyword] = []
      keyword = self.filters_difference(filters_list[:3],specified)
    
    # Treat the special set of pressure score questions
    if contained_pressure and "Crop" in original_filters:
        pressure_level = pressure_score_generator.calculate_pressure_score(original_filters["Crop"][0])
        match = [match[i] + 1 if (("pressure" in filters_list[i].keys()) and (pressure_level in filters_list[i]["pressure"])) else match[i] for i in range(len(filters_list))]
        match = [match[i] + 1 if (("pressure" in filters_list[i].keys()) and (filters_list[i]["pressure"] == [pressure_level])) else match[i] for i in range(len(filters_list))]
    sorted_docs = sorted(zip(match,docs), key=lambda pair: pair[0], reverse=True)
    #for doc in sorted_docs:
      #print(doc[1].text)
      #print(doc[1].meta)
    return sorted_docs[:3]

  def run(self, **kwargs):
    #print("Running Question Generator")
    original_filters = self.individualFiltersGenerator(kwargs["query"])
    specified = list(original_filters.keys())
    return ({"result": self.furtherQuestions(kwargs["documents"],specified,original_filters)},"output_1")

question_generator = FurtherQuestionGenerator()

In [52]:
class Result:
  outgoing_edges = 1

  def run(self,**kwargs):
    #print("Running Result")
    #print(type(kwargs["result"]))
    if type(kwargs["result"]) is list:
      docs = [doc[1] for doc in kwargs["result"]]
      return (docs,"output_1")
    else:
      return (kwargs["result"]["answer"],"output_1")

result = Result()

## Assembling into Pipeline

In [55]:
# Current approach
from haystack import Pipeline
from haystack.pipeline import JoinDocuments

# Building new pipeline with multiple retrievers
pipeline = Pipeline()
pipeline.add_node(component=es_retriever, name="ESRetriever", inputs=["Query"])
pipeline.add_node(component=dpr_retriever, name="DPRRetriever", inputs=["Query"])
pipeline.add_node(component=embedding_retriever, name="EmbeddingRetriever", inputs=["Query"])
pipeline.add_node(component=JoinDocuments(join_mode="merge"), name="JoinResults", inputs=["DPRRetriever","EmbeddingRetriever","ESRetriever"])
pipeline.add_node(component=query_classifier, name="QueryClassifier", inputs=["JoinResults"])
pipeline.add_node(component=question_generator, name="QnGenerator", inputs=["QueryClassifier.output_1"])
pipeline.add_node(component=table_retriever, name="TableRetriever", inputs=["QueryClassifier.output_2"])
pipeline.add_node(component=result, name="Result", inputs=["QnGenerator", "TableRetriever"])
pipeline.draw(path="custom_pipe.png")

# Question input goes here
question = input("What would you like to ask about?")
response = pipeline.run(query=question_parsing(question), top_k_retriever=20)

# Final answer is here 
#final_result = response.text if type(response) is Document else response["answer"]
if type(response) is list:
  print(response[0].text)
else:
  print(response)

What would you like to ask about?What to use for winter wheat?


100%|██████████| 1/1 [00:00<00:00, 497.90it/s]
02/26/2021 22:33:09 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.010s]
02/26/2021 22:33:09 - WARNING - farm.data_handler.processor -   Currently no support in InferenceProcessor for returning problematic ids
100%|██████████| 1/1 [00:00<00:00, 462.18it/s]


Which area are you in? (E.g. east/north/etc)(If not, please reply No) East
To give a better suggestion. We need to calculate the disease pressure of your current case. Please specify your choice by replying with the number accordingly.
Please choose the varieties which best describes your current case: 
1 :  KWS Extase
2 :  Dunston/Graham/KWS Zyatt/Moulton/LG Motown/LG Sundance/RGT Illustrious/KWS Firefly
3 :  Bennington/Costello/Elicit/Freiston/Gleam/KWS Crispin/KWS Siskin/Revelation/Skyfall
4 :  Crusoe/Evolution/KWS Kerrin/KWS Silverstone/KWS Trinity/LG Skyscraper/LG Spotlight/Shabras
5 :  Claire/Dickens/Elation/Grafton/JB Diego/KWS Basset/KWS Jackal/KWS Lili/LG Detroit/RGT Gravity/Savello/Viscount/Zulu
6 :  Cordiale/Gallant/KWS Barrel/Leeds/LG Jigsaw/LG Interstellar/LG Rhythm/LG Sabertooth/Myriad/Reflection/KWS Santiago/SY Loki
1
Please choose the area in the country which best describes your current case: 
1 :  East
2 :  North
3 :  South East
4 :  West
5 :  South
6 :  South West
2
